# Combinaison de DataFrames

## Guide officiel complet (Beaucoup d'exemples !)

### https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

---

In [ ]:
import numpy as np
import pandas as pd

## Concaténation

Permet de "coller" directement les DataFrames entre eux.

In [ ]:
data_one = {'A': ['A0', 'A1', 'A2', 'A3'],'B': ['B0', 'B1', 'B2', 'B3']}

In [ ]:
data_two = {'C': ['C0', 'C1', 'C2', 'C3'], 'D': ['D0', 'D1', 'D2', 'D3']}

In [ ]:
one = pd.DataFrame(data_one)

In [ ]:
two = pd.DataFrame(data_two)

In [ ]:
one

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [ ]:
two

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


### Concaténation le long des lignes

#### Axis = 0 

In [ ]:
axis0 = pd.concat([one,two],axis=0)

In [ ]:
axis0

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1
2,NaN,NaN,C2,D2
3,NaN,NaN,C3,D3


### Concaténation le long des colonnes

#### Axis = 1



In [ ]:
axis1 = pd.concat([one,two],axis=1)

In [ ]:
axis1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


### Axe 0, mais les colonnes correspondent
**Au cas où vous le voulez :**

In [ ]:
two.columns = one.columns

In [ ]:
pd.concat([one,two])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


## Fusion

### Tables de données

In [ ]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

In [ ]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [ ]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


### pd.merge()

Fusionne les DataFrames pandas sur la base des colonnes clés, de manière similaire à une jointure SQL. Résultats basés sur le paramètre **how**.

In [ ]:
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left, right, how: str = 'inner', on=None, left_on=None, right_on=None, left_index: bool = False, right_index: bool = False, sort: bool = False, suffixes=('_x', '_y'), copy: bool = True, indicator: bool = False, validate=None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame or named Series
        Object to merge with.
    how : {'left', 'right', 'outer', 'inner'}, default 'inner'
        Type of merge to be performed.
    
        * left: use only keys from left frame, similar to a SQL left outer join;
          preserve key order.
        * right: use only keys from right fra

-----

### Fusions Inner, Left, Right et Outer

#### Inner Join

Rattachement où la clé est présente dans les DEUX tables. Il ne devrait pas y avoir de NaNs à cause de la jointure, puisque par définition pour faire partie de la jointure interne, ils doivent avoir des informations dans les deux tables.
**Seuls Andrew et Bobo sont enregistrés et connectés.**

In [ ]:
# Remarquez que pd.merge ne prend pas en compte une liste comme concat.
pd.merge(registrations,logins,how='inner',on='name')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [ ]:
# Pandas est suffisamment intelligent pour déterminer la colonne clé (sans le paramètre ON) 
# si un seul nom de colonne correspond et est présent et identique dans les 2 tables.
pd.merge(registrations,logins,how='inner')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [ ]:
# Pandas signale une erreur si la colonne clé "on" ne figure pas dans les deux DataFrames.
# pd.merge(registrations,logins,how='inner',on='reg_id')

---

#### Left Join

**Rattacher ET inclure toutes les lignes de la table de gauche.**
**Montrer tous ceux qui se sont inscrits dans la table de gauche, s'ils n'ont pas d'informations de login, alors remplir avec NaN.**

In [ ]:
pd.merge(registrations,logins,how='left')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


#### Right Join

**Rattacher ET inclure toutes les lignes de la table de droite.**
**Montrez toutes les personnes qui se sont connectées dans la table de droite, si elles n'ont pas d'informations d'enregistrement, remplissez avec NaN.

In [ ]:
pd.merge(registrations,logins,how='right')

,reg_id,name,log_id
0,NaN,Xavier,1
1,1.0,Andrew,2
2,NaN,Yolanda,3
3,2.0,Bobo,4


#### Outer Join

**Correspondance de toutes les informations trouvées dans la table de gauche ou de droite**.
**Montrez à tout le monde ce qui se trouve dans la table login et dans la table registrations. Remplissez toute information manquante avec NaN**

In [ ]:
pd.merge(registrations,logins,how='outer')

,reg_id,name,log_id
0,1.0,Andrew,2.0
1,2.0,Bobo,4.0
2,3.0,Claire,NaN
3,4.0,David,NaN
4,NaN,Xavier,1.0
5,NaN,Yolanda,3.0


#### Join sur Index ou Colonne

**Utiliser des combinaisons de left_on, right_on, left_index, right_index pour fusionner une colonne ou un index l'un sur l'autre**.

In [ ]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [ ]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [ ]:
registrations = registrations.set_index("name")

In [ ]:
registrations

,reg_id
name,
Andrew,1
Bobo,2
Claire,3
David,4


In [ ]:
pd.merge(registrations,logins,left_index=True,right_on='name')

,reg_id,log_id,name
1,1,2,Andrew
3,2,4,Bobo


In [ ]:
pd.merge(logins,registrations,right_index=True,left_on='name')

,log_id,name,reg_id
1,2,Andrew,1
3,4,Bobo,2


#### Traitement des noms de colonnes clés différents dans les tables jointes

In [ ]:
registrations = registrations.reset_index()

In [ ]:
registrations

,name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [ ]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [ ]:
registrations.columns = ['reg_name','reg_id']

In [ ]:
registrations

,reg_name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [ ]:
# ERREUR
# pd.merge(registrations,logins)

In [ ]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name')

,reg_name,reg_id,log_id,name
0,Andrew,1,2,Andrew
1,Bobo,2,4,Bobo


In [ ]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name').drop('reg_name',axis=1)

,reg_id,log_id,name
0,1,2,Andrew
1,2,4,Bobo


#### Pandas marque automatiquement les colonnes dupliquées

In [ ]:
registrations.columns = ['name','id']

In [ ]:
logins.columns = ['id','name']

In [ ]:
registrations

,name,id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [ ]:
logins

,id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [ ]:
# _x pour gauche
# _y pour droite
pd.merge(registrations,logins,on='name')

,name,id_x,id_y
0,Andrew,1,2
1,Bobo,2,4


In [ ]:
pd.merge(registrations,logins,on='name',suffixes=('_reg','_log'))

,name,id_reg,id_log
0,Andrew,1,2
1,Bobo,2,4
